# Differantial Private Deep Learning

When our neural network are learning from sensitive data, that thay are only learning what they're supposed to learn from the data without accidentally learning they are not supposed to learn from the data.

The general goal of differantial privacy is to ensure that different kind of statistical analysis do not compromise the privacy.

The defination of Privacy: You will not affected, adversly on otherwise, by allowing your data to be used in any study on analysis, no matter what other studies,datasets, orinformation sources, are avaible.


Exercise
* Step one is to create our database - we're going to do this by initializing a random list of 1s and 0s (which are the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

*  I want you to create a list of every parallel database to the one currently contained in the "db" variable. Then, I want you to create a function which both:

    * creates the initial database (db)
    * creates all parallel databases


In [1]:
#function to crete a databse
import torch
def remove_index(db,index):
    """Remove the defined index from database"""
    #removing the index
    return torch.cat((db[0:index],db[index + 1:]))

def paralle_db(db):
    """Create paralel data base for a given database with using remove_index function"""
    #create the empty list for paralel databases
    paralel_dbs = list()
    #loop over the database
    for i in range(len(db)):
        #create the database for each removed index
        dbs = remove_index(db,i)
        #appande the removed databases to list of paralel databases
        paralel_dbs.append(dbs)
    return paralel_dbs

def create_and_paralel(num_enrty):
    """Create simple database for a single column and then call the parael_db to
    create paralal databases for each removed index value"""
    #create the simple database for specific number 
    db = torch.rand(num_enrty) > 0.5
    #call paralle_bd function to create paralal databases for spefic number of element
    pbds = paralle_db(db)
    return db, pbds
    

In [2]:
bd, pdbs = create_and_paralel(5000)

In [3]:
#let's check out first 6 tenser list from pdbs
print(pdbs[0:5])

[tensor([1, 0, 1,  ..., 0, 0, 0], dtype=torch.uint8), tensor([1, 0, 1,  ..., 0, 0, 0], dtype=torch.uint8), tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8), tensor([1, 1, 0,  ..., 0, 0, 0], dtype=torch.uint8), tensor([1, 1, 0,  ..., 0, 0, 0], dtype=torch.uint8)]


# Evaluating the The Privacy of the Function

*  How to query this database
* then measure the privacy of that query

We are going to compare the output of the query on the entire database, with the output of the query on each of the paralel databases. That way, we are going to how the query change, when we remove an individual from tha database.

* We are going to query the full database
* Weare going to look at quering every possible paralel database
* What's the maxmimum around that they different?

When we remove the people from this database, it changes the output of query. the output of this queryis actually conditionied directy on information from a lot of people in this databse.

Sensitivity: The maximum amount that the query changes when removing an individual from the database. it is called L1 sensitivity for simply.




In [4]:
def query(db):
    return db.sum()

In [5]:
db, pdbs = create_and_paralel(5000)

In [6]:
full_db_result = query(db)

In [7]:
sensitivy = 0
for dbs in pdbs:
    pdb_result = query(dbs)
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if (db_distance > sensitivy):
        sensitivy = db_distance
    
sensitivy

tensor(1)

In [8]:
def query(db):
    return db.float().mean()

def sensitivity(query, n_entries =1000):
    db, pdbs = create_and_paralel(n_entries)
   
    full_db_result = query(db)
    sensitivity = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > sensitivity):
            sensitivity = db_distance
    
    return sensitivity

In [9]:
sensitivity(query)

tensor(0.0005)

# Local & Global Differantial Privacy
**Local differantial privacy:** Add noise to the function dat points
**Global Differantial pricacy:** Add noise on the output query on the database

## Local Differantial Privacy

Where given a collection of individual, each individial adds noise to the their data before sending it to the statistical database itself. So, everything that gets through is alrready noised. So, the protection is happening at the local level. 

Differantial privacy always requires a form of randomness or noise added to the quey to protec from like differnatial attack.

**Ransomized Responce:** Techniquie that is used social sciences when tring to a learn about the high level trends for a taban behavior.

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

**Flip a coin 2 times.**
* If the first coin flip is heads, answer honestly
* If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained.

However, it should be noted that, especially when we only have a few samples, the this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results.



In [106]:
db, pdbs = create_and_paralel(100)
#db
first_coin_flip = (torch.rand(len(db))> 0.5).float()
#first_coin_flip
second_coin_flip = (torch.rand(len(db))> 0.5).float()
augmented_databse = (db.float() * first_coin_flip) + ((1-first_coin_flip) * second_coin_flip)
augmented_databse
torch.mean(augmented_databse).float() *2 -0.5

tensor(0.6200)

**in the video**

$$augmented\_databse\_mean = (true\_mean *noise) + (noise\_dist\_mean*(1-noise))$$

**it is also equel to**

$$augmented\_databse\_mean = (true\_mean *(1-noise)) + (noise\_dist\_mean* noise)$$

**changing the $noise$ and $(1-noise)$ are the same...**

So, if we try to get $true\_mean$ here

**Note: $augmented\_databse\_mean is alse called sk_result which we are calculating before. So, I am going to continue with sk_result**

$$ sk\_result =(true\_mean *(1-noise)) + (noise\_dist\_mean* noise)$$

we want to $true\_mean$ alone. So, I want to take (1-noise) paranteses for both term

$$sk\_result = (1-noise)(true\_mean + noise\_dist\_mean* (\frac{noise}{1-noise}) )$$

$$ \frac{sk\_result}{(1-noise)} =  true\_mean + noise\_dist\_mean * (\frac{noise}{1-noise})$$

 
 $$true\_mean = \frac{sk\_result}{(1-noise)} - noise\_dist\_mean * (\frac{noise}{1-noise}) $$
 
**S0, we can take the $\frac{noise}{(1-noise)}$ parantehesis**

$$true\_mean = (\frac{noise}{(1-noise)}) * (\frac{sk\_result}{noise} - noise\_dist\_mean)  $$

we know that $noise\_dist\_mean = 0.5$

$$true\_mean = (\frac{noise}{(1-noise)}) * (\frac{sk\_result}{noise} - 0.5)  $$

$$true\_mean = ((sk\_learn / noise) - 0.5)* noise/(1-noise) $$







In [109]:
def queryy(db):
    true_mean = db.float().mean()
    #flip the coins
    first_coin_flip = (torch.rand(len(db)) > 0.5).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()
    #augmented_databse 
    augmented_databse = (first_coin_flip * db.float()) + ((1 - first_coin_flip) * second_coin_flip) 
    augmented_databse_mean = augmented_databse.float().mean()
    return true_mean, augmented_databse_mean

    

In [112]:
db, pdbs =create_and_paralel(1000)
true_mean, augmented_databse_mean =  queryy(db)
print("With Noise : " +str(true_mean))
print("Witout Noise: "+ str(augmented_databse_mean))

With Noise : tensor(0.4720)
Witout Noise: tensor(0.4640)


In [115]:
def query_noise(db, noise=0.2):
    true_mean = db.float().mean()
    #flip the coins
    first_coin_flip = (torch.rand(len(db)) > noise).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()
    #augmented_databse 
    augmented_databse = (first_coin_flip * db.float()) + ((1 - first_coin_flip) * second_coin_flip) 
    sk_result = augmented_databse.float().mean()
    private_result = ((sk_result/noise)- 0.5)*( noise*(1-noise))
    return sk_result, private_result
    

In [117]:
db, pdbs =create_and_paralel(1000)
true_mean, augmented_databse_mean =  query_noise(db, noise=0.2)
print("With Noise : " +str(true_mean))
print("Witout Noise: "+ str(augmented_databse_mean))

With Noise : tensor(0.5110)
Witout Noise: tensor(0.3288)


#### The Formal Definition of Differential Privacy
How much noise should add in the query?

we are going to create a database with our quey then analyze it with the formal  defination of differential privacy:
```python
def query(db, epsilon=1):
    scale = 1/epsilon
    s = np.random.laplace(0, scale, 1)
    #adding the privacy code outside og sum
    return (torch.sum(db.float()) + s)[0]
   
```
We are going to add noise to our function and we have two differant kinds of noise that we can add, Laplacian noise, Gaussian noise,

The formula is a contraint so that you can analyze a query with noise and know whether or not this query is leaking too much information.

Global differantial privacy  instead of adding noise to individual data points applies noise to the output of a function on data points. 

Advantage of this, you can often add a lot less noise and get a much more accurate result if youwait to add the noise until after you are  computed a function.


How do we actually measure how much privacy is being leaked inside of a diffferantially private algorithm?


**Differantial Privacy:** 

A normalized algorithm $\mathcal{M}$ with domain $\mathbb{N}^{|x|}$ is ($\epsilon, \, \delta$)- differantially private if for all $\mathcal{S}\subseteq$ Range($\mathcal{M}$) and for all $x,y \in \mathbb{N}^{|x|}$ such that $||x-y||_1 \le 1$

$$Pr[\mathcal{M}(x) \in \mathcal{S}] \le exp(\epsilon) Pr[\mathcal{M}(y) \in \mathcal{S}] + \delta$$


M being a randomized is:
```python
def M(db):
    return query(db) +noise 
```
It's some  sort of function on a database with some sort of noise added to it. This would be globally diffrential private algorithm (query(db) +noise) or this could have been inside the database query(db+noise) which makes is local differantial private. So, where the noise is actually added is does not matter, the exact mechanism is nonspecific in the formula. So, this could be any function, we just know that it has the ability to query a databse and makes randomized in some way. And it has a output domain $\mathcal{N}(x)$, natural numbers, meaning it could be a histogram over certain entries in the database . So, it's discretized in this particular case, and we're saying that this is epsilon delta differantially private if for all the potential things that $\mathcal{M}$  could predic, so, all potentiall outputs of $\mathcal{M}$ for all potential inputs(x,y) they are paralel databases. Menaing he $Pr[\mathcal{M}(x) \in \mathcal{S}] \le exp(\epsilon) Pr[\mathcal{M}(y) \in \mathcal{S}] + \delta$ equation is TRUE. 

Actually,$x,y$ are histograms over pair databases, and this two histograms maximum ditance could be 1, meaning they have jush have one different entries in their databases.

$$Pr[\mathcal{M}(x) \in \mathcal{S}] \le exp(\epsilon) Pr[\mathcal{M}(y) \in \mathcal{S}] + \delta$$

x. takes our mechanism and running it over histogram and y is doing the same thing for any paralel database with one entry missing. So, this threshold has to be true for all databeses and its returning something in $\mathcal{S}$, S is one of possible thing that it could be predicting in the range of $\mathcal{M}$, 

The core here how different are these two distribution is. How much does it changes is measured by two prameters, $\epsilon, \delta$. $exp(\epsilon)$ is show how diffrent these two distrubutin is, 

Epsilon Zero: If a query satisfied this inequality where epsilon was set to 0, then that would mean that the query for all parallel databases outputed the exact same value as the full database. As you may remember, when we calculated the "threshold" function, often the Sensitivity was 0. In that case, the epsilon also happened to be zero.

Epsilon One: If a query satisfied this inequality with epsilon 1, then the maximum distance between all queries would be 1 - or more precisely - the maximum distance between the two random distributions M(x) and M(y) is 1 (because all these queries have some amount of randomness in them, just like we observed in the last section).



Now, $\delta$: is the probabbilty that you will accidenty leak, more information then $\epsilon$ claims that you will leak. it is usually very small number

# Lesson: How To Add Noise for Global Differential Privacy

In this lesson, we're going to learn about how to take a query and add varying amounts of noise so that it satisfies a certain degree of differential privacy. In particular, we're going to leave behind the Local Differential privacy previously discussed and instead opt to focus on Global differential privacy. 

So, to sum up, this lesson is about adding noise to the output of our query so that it satisfies a certain epsilon-delta differential privacy threshold.

There are two kinds of noise we can add - Gaussian Noise or Laplacian Noise. Generally speaking Laplacian is better, but both are still valid. Now to the hard question...

### How much noise should we add?

The amount of noise necessary to add to the output of a query is a function of four things:

- the type of noise (Gaussian/Laplacian)
- the sensitivity of the query/function
- the desired epsilon (ε)
- the desired delta (δ)

Thus, for each type of noise we're adding, we have different way of calculating how much to add as a function of sensitivity, epsilon, and delta. We're going to focus on Laplacian noise. Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now.


In [120]:
import numpy as np

In [121]:
def laplace_query_sum(db, epsilon=1):
    #b = sensitiviyu/epsilon
    scale = 1/ epsilon
    s = torch.tensor(np.random.laplace(0, scale,1))
    return (torch.sum(db.float()) + s)    

In [122]:
db, pdbs = create_and_paralel(100)

In [125]:
#true results
torch.sum(db.float())

tensor(42.)

In [129]:
sum_laplace_query = laplace_query_sum(db, epsilon=1)
sum_laplace_query

tensor([42.5033], dtype=torch.float64)

# Differential Privacy for Deep Learning


First we're going to consider a scenario - you work for a hospital and you have a large collection of images about your patients. However, you don't know what's in them. You would like to use these images to develop a neural network which can automatically classify them, however since your images aren't labeled, they aren't sufficient to train a classifier.

However, being a cunning strategist, you realize that you can reach out to 10 partner hospitals which DO have annotated data. It is your hope to train your new classifier on their datasets so that you can automatically label your own. While these hospitals are interested in helping, they have privacy concerns regarding information about their patients. Thus, you will use the following technique to train a classifier which protects the privacy of patients in the other hospitals.

1) You'll ask each of the 10 hospitals to train a model on their own datasets (All of which have the same kinds of labels)
2) You'll then use each of the 10 partner models to predict on your local dataset, generating 10 labels for each of your datapoints
3) Then, for each local data point (now with 10 labels), you will perform a DP query to generate the final true label. This query is a "max" function, where "max" is the most frequent label across the 10 labels. We will need to add laplacian noise to make this Differentially Private to a certain epsilon/delta constraint.
4) Finally, we will retrain a new model on our local dataset which now has labels. This will be our final "DP" model.
So, let's walk through these steps. I will assume you're already familiar with how to train/predict a deep neural network, so we'll skip steps 1 and 2 and work with example data. We'll focus instead on step 3, namely how to perform the DP query for each example using toy data.

So, let's say we have 10,000 training examples, and we've got 10 labels for each example (from our 10 "teacher models" which were trained directly on private data). Each label is chosen from a set of 10 possible labels (categories) for each image.

In [4]:
import numpy as np

In [4]:
num_teachers =10 # we are going to work with 10 hospital
num_examples = 1000 # the size of our dataset
num_labels =10 #number 0f classifier for our dataset

In [8]:
#let's create sentetik dataset
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int).transpose(1,0)
preds

array([[4, 6, 1, ..., 9, 6, 5],
       [6, 8, 8, ..., 1, 1, 4],
       [9, 5, 7, ..., 0, 5, 4],
       ...,
       [5, 4, 0, ..., 2, 2, 8],
       [6, 9, 6, ..., 9, 3, 3],
       [8, 9, 7, ..., 3, 0, 5]])

In [12]:
new_labes = list()
for an_image in preds:
    #counting the most frequensist teachers which appear in the array
    label_counts = np.bincount(an_image, minlength=num_labels)
    
    #create our privacy
    epsilon =0.1
    beta = 1/epsilon
    
    for i in range(len(label_counts)):
        label_counts[i] = np.random.laplace(0, beta, 1)
    new_label = np.argmax(label_counts)
    new_labes.append(new_label)
    

[0, 1, 8, 8, 4, 8, 1, 6, 8, 7, 6, 5, 7, 5, 4, 6, 4, 8, 4, 0, 4, 9, 5, 9, 5, 1, 9, 4, 4, 0, 4, 3, 1, 2, 1, 4, 8, 6, 2, 0, 8, 7, 4, 4, 5, 6, 2, 3, 9, 7, 3, 7, 2, 6, 8, 7, 2, 8, 5, 5, 8, 8, 3, 9, 8, 8, 5, 1, 2, 4, 8, 4, 9, 0, 0, 6, 0, 9, 4, 1, 6, 0, 1, 3, 9, 2, 4, 8, 3, 6, 0, 0, 8, 4, 7, 8, 3, 8, 0, 4, 1, 1, 7, 6, 7, 0, 1, 0, 8, 1, 9, 8, 0, 0, 0, 6, 3, 0, 2, 5, 8, 3, 6, 2, 8, 8, 7, 8, 9, 3, 0, 6, 4, 2, 3, 6, 2, 0, 5, 6, 6, 0, 6, 8, 5, 1, 3, 3, 6, 1, 8, 3, 0, 1, 3, 5, 5, 6, 9, 3, 6, 6, 1, 1, 4, 4, 7, 5, 4, 7, 5, 7, 9, 9, 6, 2, 7, 2, 4, 6, 2, 9, 0, 9, 0, 4, 1, 9, 7, 7, 6, 8, 5, 3, 2, 0, 3, 6, 0, 3, 0, 5, 0, 2, 4, 7, 1, 8, 1, 5, 5, 8, 2, 9, 9, 2, 2, 4, 2, 6, 9, 7, 7, 0, 4, 0, 0, 2, 1, 6, 6, 9, 2, 7, 3, 8, 4, 0, 8, 4, 1, 9, 8, 7, 8, 4, 9, 0, 9, 4, 4, 2, 5, 1, 3, 9, 9, 0, 2, 6, 7, 2, 3, 1, 3, 2, 4, 9, 1, 4, 2, 0, 2, 4, 9, 6, 8, 5, 6, 2, 1, 0, 0, 5, 6, 3, 5, 3, 8, 5, 7, 9, 3, 6, 8, 3, 9, 6, 9, 5, 3, 4, 8, 4, 5, 1, 9, 1, 5, 0, 2, 8, 9, 3, 1, 1, 1, 0, 1, 1, 8, 0, 9, 1, 7, 9, 0, 1, 7, 3, 7, 4, 5, 

# PATE ANALYSIS 

In [2]:
from syft.frameworks.torch.differential_privacy import  pate


In [6]:
num_teachers, num_examples, num_labels = (100, 100, 10)
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int) #fake preds
indices = (np.random.rand(num_examples) * num_labels).astype(int) # true answers

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)

assert data_dep_eps < data_ind_eps

In [7]:
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 11.756462732485105


In [8]:
#make our result more similiar and see the results
#making 10 example to be force 0
preds[:, 0:10] *=0
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 1.52655213289881


In [10]:
#make our result more similiar and see the results
#making 10 example to be force 0
preds[:, 0:50] *=0
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments=20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 0.9029013677789843


**The more predictions agree with each other the tigher an epsilon value we can set.**